# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


# Import Graph Querying Library

In [2]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

In [3]:
schema = service.get_kh_edge_list()

# Create an Overview/Exploration Widget

In [21]:
from kyurem import Explorer


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
    }

def focus(state, node, panel):
    # node can be in one of two formats:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    # node : { "node_label": str, "title": str, "uuid": str }
    
    # Since the service functions use the first format, node must first
    # be converted to the proper format
    if "title" in node:
        node = {
            "node_label": node["node_label"],
            "node_property": "title",
            "node_property_value": node["title"]
        }
        
    # TODO: Return nodes in a consistent format from the service
    #       functions to streamline the above
    
    
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        # TODO: Accept a consistent input format (e.g., same as get_node_neighborhood)
        data["children"] = service.get_children_node_distributions(
            node["node_label"], 
            node["node_property"], 
            node["node_property_value"]
        )

    # Use a list comprehension to convert the return format of the relation_dist 
    # to a bar-chart format
    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type}
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    # TODO: Move conversion code into service
    
    return data
  
# Create widget 
reload_bundle()
explorer = Explorer({
        "init": init,
        "focus": focus
    }, schema)

explorer.show()

component(7efc54619dc0, self=<kyurem.widgets.ReducerWidget.ReducerWidget object at 0x7efc545867c0>)

LayoutWidget(Layout(component(7efc54619dc0, self=<kyurem.widgets.ReducerWidget.ReducerWidget object at 0x7efc5…

In [22]:
from kyurem.utils.mdump import mdump
print(mdump(explorer.history, 3))

[ {
    "action": {
      "name": "init",
      "args": (),
      "kwargs": {},
    },
    "state": {
      "did_init": True,
      "data": { ...4 items... },
      "is_loading": False,
      "focus_node": { ...3 items... },
      "focus_panel": None,
    },
  } ]


In [6]:
# def filter_by_relation(state, type, direction):
#     if state.nodelabel and state.nodetitle:
#         node = {
#             "node_label": state.nodelabel,
#             "node_property": "title",
#             "node_property_value": state.nodetitle,
#         }
#     else:
#         node = None

#     result = service.get_relation_neighborhood(node, {"type": type, "direction": direction})
#     data = {}
#     data['schema'] = result["schema"]
#     if not node:
#         data['children'] = [
#             {"x": x, "y": y} for x, y in result["node_dist"].items()
#         ]
#     return data

# View 0: Send nodes rather than labels in schema

In [7]:
schema

[{'emphasis': 'yes',
  'label': 'EXTRACTED_BY',
  'source': {'node_label': 'anyConcept',
   'node_property': 'title',
   'node_property_value': 'anyConcept'},
  'target': {'node_label': 'EXTRACTOR',
   'node_property': 'title',
   'node_property_value': 'EXTRACTOR'},
  'weight': 1},
 {'emphasis': 'yes',
  'label': 'EXTRACTED_BY',
  'source': {'node_label': 'attribute',
   'node_property': 'title',
   'node_property_value': 'attribute'},
  'target': {'node_label': 'EXTRACTOR',
   'node_property': 'title',
   'node_property_value': 'EXTRACTOR'},
  'weight': 1},
 {'emphasis': 'yes',
  'label': 'EXTRACTED_BY',
  'source': {'node_label': 'companySector',
   'node_property': 'title',
   'node_property_value': 'companySector'},
  'target': {'node_label': 'EXTRACTOR',
   'node_property': 'title',
   'node_property_value': 'EXTRACTOR'},
  'weight': 1},
 {'emphasis': 'yes',
  'label': 'EXTRACTED_BY',
  'source': {'node_label': 'occupation',
   'node_property': 'title',
   'node_property_value': 

# View 0: Click a node on minimap

In [8]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'occupation'}

service.get_children_node_distributions('occupation', 'title', 'Healthcare Occupations')

[{'x': 'Pharmacists', 'y': 2},
 {'x': 'Veterinarians', 'y': 2},
 {'x': 'Medical Affairs Occupations', 'y': 2},
 {'x': 'Dentists', 'y': 2},
 {'x': 'Community Health & Wellbeing Occupations', 'y': 3},
 {'x': 'Pharmacy Occupations', 'y': 4},
 {'x': 'Healthcare Assisting Occupations', 'y': 5},
 {'x': 'Care Workers', 'y': 5},
 {'x': 'Medical Device Occupations', 'y': 5},
 {'x': 'Veterinary Occupations', 'y': 5},
 {'x': 'Healthcare Science Practitioners & Assistants', 'y': 5},
 {'x': 'Healthcare Technicians', 'y': 6},
 {'x': 'Dental Occupations', 'y': 6},
 {'x': 'Alternative Medicine Occupations', 'y': 8},
 {'x': 'Healthcare Management Occupations', 'y': 8},
 {'x': 'Therapy Occupations ', 'y': 8},
 {'x': 'Mental & Behavioral Health Occupations', 'y': 9},
 {'x': 'Physicians & Physician Assistants', 'y': 9},
 {'x': 'Technical Healthcare Occupations', 'y': 9},
 {'x': 'Ophthalmology Occupations', 'y': 9},
 {'x': 'Healthcare Administrative Support Occupations', 'y': 10},
 {'x': 'Nurses', 'y': 15}

# View 0: Click a bar on node_dist

In [9]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Allied Health Occupations'}

In [10]:
nn = service.get_node_neighborhood(node)
nn

{'relation_dist': {'in': [{'count': 1, 'label': 'specialization'},
   {'count': 32, 'label': 'generalization'}],
  'out': [{'count': 1, 'label': 'generalization'},
   {'count': 1, 'label': 'EXTRACTED_BY'},
   {'count': 1, 'label': 'EXTRACTED_FROM'},
   {'count': 32, 'label': 'specialization'}]},
 'schema': [{'emphasis': 'yes',
   'label': 'specialization',
   'source': {'node_label': 'occupation',
    'node_property': 'title',
    'node_property_value': 'Allied Health Occupations'},
   'target': {'node_label': 'occupation',
    'node_property': 'title',
    'node_property_value': 'Healthcare Occupations'},
   'weight': 1},
  {'emphasis': 'yes',
   'label': 'generalization',
   'source': {'node_label': 'occupation',
    'node_property': 'title',
    'node_property_value': 'Allied Health Occupations'},
   'target': {'node_label': 'occupation',
    'node_property': 'title',
    'node_property_value': 'Patient Care Advocates'},
   'weight': 1},
  {'emphasis': 'yes',
   'label': 'generaliza

# View 0: Click a bar on relation dist

In [11]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Fire Alarm Technicians'}

relation = {}
relation['type'] = 'has_core_skill'
relation['direction'] = 'out'

In [12]:
service.get_relation_neighborhood(node, relation)

RetryError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /relation_neighborhood (Caused by ResponseError('too many 500 error responses'))

In [ ]:
service.get_relation_neighborhood(None, relation)

# View 1: Click a node on zoomed in graph

# View 1: Click a node on node_dist

# View 1: Click a node on relation dist